# Import libraries

In [ ]:
import os
from dotenv import load_dotenv
from pathlib import Path
from config.load_config import load_config
import requests
from IPython.display import Image, display
from ultralytics import YOLO
import logging
from datetime import datetime

# Create enviroment variables

In [ ]:
load_dotenv()
cfg = load_config()

In [ ]:
PEXELS_API_KEY = os.getenv("PEXELS_API_KEY")
if not PEXELS_API_KEY:
    raise ValueError("PEXELS_API_KEY is not set in the environment variables.")
else:
    print("PEXELS_API_KEY is set.")
BASE_DIR = Path(cfg["base_dir"]).resolve()
print(BASE_DIR)
INPUT_DIR = Path(BASE_DIR / cfg["input_dir"]).resolve()
print(INPUT_DIR)
OUTPUT_DIR = Path(BASE_DIR / cfg["output_dir"]).resolve()
print(OUTPUT_DIR)
MODEL_DIR = Path(BASE_DIR / cfg["model_dir"]).resolve()
print(MODEL_DIR)
LOGS_DIR = Path(BASE_DIR / cfg["logs_dir"]).resolve()
print(LOGS_DIR)

# Create folders

In [ ]:
INPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)
LOGS_DIR.mkdir(parents=True, exist_ok=True)

# Setup logging

In [ ]:
LOG_FILE = LOGS_DIR / f"app_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.FileHandler(LOG_FILE, mode="a"), logging.StreamHandler()],
)

# Download test images through API

In [ ]:
# Fetch images from Pexels API
headers = {"Authorization": PEXELS_API_KEY}
params = {"query": "picnic", "per_page": 5}
response = requests.get(
    "https://api.pexels.com/v1/search", headers=headers, params=params
)

In [ ]:
for photo in response.json()["photos"]:
    url = photo["src"]["original"]
    filename = url.split("/")[-1].split("?")[0]
    path = INPUT_DIR / filename
    img = requests.get(url)
    with open(path, "wb") as f:
        f.write(img.content)
    print(f"Downloaded {filename}")

In [ ]:
# View test images
for image in INPUT_DIR.iterdir():
    if image.suffix.lower() in [".jpg", ".jpeg", ".png"]:
        print(f"Image: {image.name}")
        display(Image(filename=image, width=300, height=300))

# Load model

In [ ]:
model_path = MODEL_DIR / cfg["model_name"]

In [ ]:
model = YOLO(model_path)
logging.info(f"Model loaded from: {model_path}\nModel info:{model.info()}")

# Test model

In [ ]:
results = model.predict(
    source=INPUT_DIR,
    save=True,
    save_txt=True,
    save_conf=True,
    project=OUTPUT_DIR,
    name="predictions",
    #conf=cfg["confidence_threshold"],
    conf=0.2,
    iou=0.45,
    max_det=15,
    exist_ok=True,
)
PREDICTIONS_DIR = OUTPUT_DIR / "predictions"

In [ ]:
# View test images
for image in PREDICTIONS_DIR.iterdir():
    if image.suffix.lower() in [".jpg", ".jpeg", ".png"]:
        print(f"Image: {image.name}")
        display(Image(filename=image, width=800, height=800))

# Results
* We can see the model is good at detecting persons
* Other objects are missclassified or the model isn't so secure about it
* For example, ChatGPT mentioned that YOLOv8 does not have the class "basket" so identifies it as a vase

# Actions to improve model
* Inspect training labels
* Check class definitions
* Add diferent images and use augmentation to improve dataset quality
* Fine-Tune the model
* Evaluate class-level performance (mAP)
* Use custom class filtering (can be pre or post prediction)

# Next steps
* At this stage, the primary goal of the project is to deploy the model using Docker rather than enchancing its prediction accuracy. Accordingly, we will advance from the testing phase tot he deployment process.